<h2>Reworked Super stars to match format of Undervalued</h2>

In [0]:
ATTRIBUTES = {
  'game_ID' : 1,
  'date' : 2,
  'player_ID' : 3,
  'player' : 4,
  'team' : 5,
  'position' : 9,
  'salary' : 11,
  'points' : 13
}

INDICES = [val for key, val in ATTRIBUTES.items()]

CURRENT_DATE = '2019-10-05'

COMPOSITION = {
  'C' : 2,
  'G' : 2, 
  'D' : 2,
  'W' : 3
}

BUDGET = 50000

TEAM = []

In [0]:
# File location and type
file_location = "/FileStore/tables/DataTable.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

#display(df)

In [0]:
RDD = df.rdd.map(list)
RDD.cache()

Out[5]: PythonRDD[46] at RDD at PythonRDD.scala:58

In [0]:
# extract only relevant columns
RDD_filtered_by_column = RDD.map(lambda entry: [entry[i] for i in INDICES])

# update attribute indices
ATTRIBUTES = {
  'game_ID' : 0,
  'date' : 1,
  'player_ID' : 2,
  'player' : 3,
  'team' : 4,
  'position' : 5,
  'salary' : 6,
  'points' : 7
}

In [0]:
# filter out rows that contain invalid 'position' and 'points' entries
RDD_cleaned = RDD_filtered_by_column.filter(lambda entry: entry[ATTRIBUTES['position']] != "#N/A" 
                                          and entry[ATTRIBUTES['points']] != "#N/A" 
                                          and entry[ATTRIBUTES['salary']] != "#N/A")
#RDD_cleaned.collect()

In [0]:
# group rows by 'player_ID'
RDD_paired_by_player = RDD_cleaned.map(lambda entry: [entry[ATTRIBUTES['player_ID']], entry])
RDD_grouped_by_player = RDD_paired_by_player.groupByKey().mapValues(list).map(lambda entry: list(entry))
#RDD_grouped_by_player.collect()

In [0]:
# sort rows by 'date' for each player
RDD_sorted_by_date = RDD_grouped_by_player.map(lambda entry: [entry[0], sorted(entry[1], key = lambda x: x[ATTRIBUTES['date']])])
#RDD_sorted_by_date.collect()

In [0]:
def compute_averagePoints(entry):
  """
    Returns the following attributes for each player (for each game):
    
      - game_ID
      - most recent game date
      - ID
      - name
      - team
      - position
      - salary in most recent game
      - points in most recent game
      - running average of points up to most recent game
      
    ** Assumes that entries for each player are sorted by date.
    
  """
  ID, rows = tuple(entry)
  count = 0
  total_points = 0
  total_cost = 0
  
  for row in rows:
    count += 1
    total_points += float(row[ATTRIBUTES['points']])
    row.append(float(total_points) / count)
    row[ATTRIBUTES['salary']] = int(row[ATTRIBUTES['salary']])
    
  return [ID, rows]

RDD_average_points = RDD_sorted_by_date.map(compute_averagePoints)

# add attribute-index mappings
ATTRIBUTES['average_points'] = 8
#RDD_average_points.collect()


In [0]:
def filter_by_current_date(entry):
  
  ID, rows = tuple(entry)
  new_rows = []
  
  for row in rows:
    if row[ATTRIBUTES['date']] < CURRENT_DATE:
      new_rows.append(row)
  
  if len(new_rows):
    return [ID, new_rows[-1]]
  else:
    return [ID, []]
  
RDD_filtered_by_date_super_stars = RDD_average_points.map(filter_by_current_date).filter(lambda x:x[1] != [])
#print(RDD_filtered_by_date.collect())

<h2>Order by average points</h2>

In [0]:
def find_super_stars(entry):
  ID, rows = tuple(entry)
  superStars = []
  
  nStars = int(len(rows)/10)
  
  i = 0
  while i < nStars:
    superStars.append(rows[i])
    i += 1
  
  return [ID, superStars]

    

RDD_paired_by_position_super_stars = RDD_filtered_by_date_super_stars.map(lambda entry: [entry[1][ATTRIBUTES['position']], entry])
RDD_grouped_by_position_super_stars = RDD_paired_by_position_super_stars.groupByKey().mapValues(list).map(lambda entry: list(entry))
RDD_sorted_by_average_points_scored = RDD_grouped_by_position_super_stars.map(
                                    lambda entry: [entry[0], sorted(entry[1], 
                                                                    key = lambda x: -x[1][ATTRIBUTES['average_points']])])

RDD_super_stars = RDD_sorted_by_average_points_scored.map(find_super_stars)
RDD_super_stars.collect()

Out[58]: [['C',
 [[2562637,
 ['43741-WPG@NYR',
 '2019-10-03',
 2562637,
 'Mika Zibanejad',
 'New York Rangers',
 'C',
 6300,
 43.1,
 43.1]],
 [4024123,
 ['43742-TOR@CBJ',
 '2019-10-04',
 4024123,
 'Auston Matthews',
 'Toronto Maple Leafs',
 'C',
 7900,
 21.4,
 26.95]],
 [3041996,
 ['43741-CGY@COL',
 '2019-10-03',
 3041996,
 'Sean Monahan',
 'Calgary Flames',
 'C',
 6000,
 24.5,
 24.5]],
 [3041969,
 ['43741-CGY@COL',
 '2019-10-03',
 3041969,
 'Nathan MacKinnon',
 'Colorado Avalanche',
 'C',
 7300,
 21.6,
 21.6]],
 [2563036,
 ['43741-FLA@TB',
 '2019-10-03',
 2563036,
 'Vincent Trocheck',
 'Florida Panthers',
 'C',
 5800,
 20.0,
 20.0]],
 [2593311,
 ['43741-MTL@CAR',
 '2019-10-03',
 2593311,
 'Erik Haula',
 'Carolina Hurricanes',
 'C',
 4600,
 19.5,
 19.5]],
 [5161,
 ['43741-MIN@NSH',
 '2019-10-03',
 5161,
 'Matt Duchene',
 'Nashville Predators',
 'C',
 5600,
 19.5,
 19.5]],
 [3112266,
 ['43742-VGS@SJ',
 '2019-10-04',
 3112266,
 'Tomas Nosek',
 'Vegas Golden Knights',
 'C',
 2600,
 31.3,
 17.9]],
 [2563057,
 ['43742-VGS@SJ',
 '2019-10-04',
 2563057,
 'William Karlsson',
 'Vegas Golden Knights',
 'C',
 5800,
 8.5,
 17.15]],
 [5731,
 ['43741-ARI@ANA',
 '2019-10-03',
 5731,
 'Derek Stepan',
 'Arizona Coyotes',
 'C',
 4800,
 15.8,
 15.8]],
 [4024989,
 ['43741-WPG@NYR',
 '2019-10-03',
 4024989,
 'Brett Howden',
 'New York Rangers',
 'C',
 3100,
 15.6,
 15.6]]]],
 ['W',
 [[3114727,
 ['43740-VAN@EDM',
 '2019-10-02',
 3114727,
 'Leon Draisaitl',
 'Edmonton Oilers',
 'W',
 7000,
 34.0,
 34.0]],
 [2976860,
 ['43740-VAN@EDM',
 '2019-10-02',
 2976860,
 'Tanner Pearson',
 'Vancouver Canucks',
 'W',
 4000,
 29.8,
 29.8]],
 [3899938,
 ['43741-CGY@COL',
 '2019-10-03',
 3899938,
 'Mikko Rantanen',
 'Colorado Avalanche',
 'W',
 6900,
 29.3,
 29.3]],
 [2968772,
 ['43741-MIN@NSH',
 '2019-10-03',
 2968772,
 'Filip Forsberg',
 'Nashville Predators',
 'W',
 6500,
 26.0,
 26.0]],
 [2501107,
 ['43742-TOR@CBJ',
 '2019-10-04',
 2501107,
 'Cam Atkinson',
 'Columbus Blue Jackets',
 'W',
 6700,
 23.8,
 23.8]],
 [2563026,
 ['43742-WPG@NJ',
 '2019-10-04',
 2563026,
 'Blake Coleman',
 'New Jersey Devils',
 'W',
 4000,
 23.0,
 23.0]],
 [3149827,
 ['43741-BUF@PIT',
 '2019-10-03',
 3149827,
 'Conor Sheary',
 'Buffalo Sabres',
 'W',
 4100,
 22.8,
 22.8]],
 [5227,
 ['43741-MTL@CAR',
 '2019-10-03',
 5227,
 'Tomas Tatar',
 'Montreal Canadiens',
 'W',
 4500,
 22.3,
 22.3]],
 [3899937,
 ['43742-TOR@CBJ',
 '2019-10-04',
 3899937,
 'Mitch Marner',
 'Toronto Maple Leafs',
 'W',
 7000,
 33.6,
 21.55]],
 [2590379,
 ['43741-CGY@COL',
 '2019-10-03',
 2590379,
 'Joonas Donskoi',
 'Colorado Avalanche',
 'W',
 3100,
 21.5,
 21.5]],
 [2500990,
 ['43742-VGS@SJ',
 '2019-10-04',
 2500990,
 'Reilly Smith',
 'Vegas Golden Knights',
 'W',
 5600,
 12.8,
 21.4]],
 [3114741,
 ['43742-WPG@NJ',
 '2019-10-04',
 3114741,
 'Nikolaj Ehlers',
 'Winnipeg Jets',
 'W',
 5900,
 8.0,
 20.75]],
 [3961,
 ['43742-WPG@NJ',
 '2019-10-04',
 3961,
 'Blake Wheeler',
 'Winnipeg Jets',
 'W',
 6100,
 6.0,
 20.65]],
 [5556,
 ['43741-MIN@NSH',
 '2019-10-03',
 5556,
 'Austin Watson',
 'Nashville Predators',
 'W',
 2900,
 19.7,
 19.7]],
 [2590389,
 ['43741-FLA@TB',
 '2019-10-03',
 2590389,
 'Ondrej Palat',
 'Tampa Bay Lightning',
 'W',
 3600,
 19.5,
 19.5]],
 [2593315,
 ['43741-MIN@NSH',
 '2019-10-03',
 2593315,
 'Jason Zucker',
 'Minnesota Wild',
 'W',
 4800,
 19.5,
 19.5]],
 [3120307,
 ['43741-MIN@NSH',
 '2019-10-03',
 3120307,
 'Viktor Arvidsson',
 'Nashville Predators',
 'W',
 7400,
 19.5,
 19.5]],
 [5545,
 ['43742-VGS@SJ',
 '2019-10-04',
 5545,
 'Mark Stone',
 'Vegas Golden Knights',
 'W',
 7200,
 19.3,
 17.8]],
 [5428,
 ['43742-WPG@NJ',
 '2019-10-04',
 5428,
 'Taylor Hall',
 'New Jersey Devils',
 'W',
 7500,
 17.5,
 17.5]],
 [2592095,
 ['43741-MTL@CAR',
 '2019-10-03',
 2592095,
 'Teuvo Teravainen',
 'Carolina Hurricanes',
 'W',
 4700,
 17.5,
 17.5]],
 [3114747,
 ['43742-WSH@NYI',
 '2019-10-04',
 3114747,
 'Jakub Vrana',
 'Washington Capitals',
 'W',
 3800,
 14.3,
 16.9]]]],
 ['D',
 [[2562605,
 ['43741-MTL@CAR',

In [0]:
def find_available_players(entry):
  
  ID, rows = tuple(entry)
  
  for row in rows:
    if row[ATTRIBUTES['date']] == CURRENT_DATE:
      return True
    
  return False
  
RDD_available_superstars = RDD_grouped_by_player.filter(find_available_players).map(lambda x: x[0])

In [0]:
def filter_by_availability(entry):
  
  position, players = tuple(entry)
  
  for player in players:
    ID, _ = tuple(player)
    if RDD_available_players.filter(lambda player: player == ID):
      return True
    
  return False

RDD_super_stars_filtered_by_availability = RDD_sorted_by_average_points_scored.filter(lambda entry: entry)
#RDD_super_stars_filtered_by_availability.collect()

<h2>Find the available superstars </h2>

In [0]:
def find_super_stars(entry):
  ID, rows = tuple(entry)
  superStars = []
  
  nStars = int(len(rows)/10)
  
  i = 0
  while i < nStars:
    superStars.append(rows[i])
    i += 1
  
  return [ID, superStars]

RDD_super_stars = RDD_super_stars_filtered_by_availability.map(find_super_stars)
RDD_super_stars.collect()

Out[61]: [['C',
 [[2562637,
 ['43741-WPG@NYR',
 '2019-10-03',
 2562637,
 'Mika Zibanejad',
 'New York Rangers',
 'C',
 6300,
 43.1,
 43.1]],
 [4024123,
 ['43742-TOR@CBJ',
 '2019-10-04',
 4024123,
 'Auston Matthews',
 'Toronto Maple Leafs',
 'C',
 7900,
 21.4,
 26.95]],
 [3041996,
 ['43741-CGY@COL',
 '2019-10-03',
 3041996,
 'Sean Monahan',
 'Calgary Flames',
 'C',
 6000,
 24.5,
 24.5]],
 [3041969,
 ['43741-CGY@COL',
 '2019-10-03',
 3041969,
 'Nathan MacKinnon',
 'Colorado Avalanche',
 'C',
 7300,
 21.6,
 21.6]],
 [2563036,
 ['43741-FLA@TB',
 '2019-10-03',
 2563036,
 'Vincent Trocheck',
 'Florida Panthers',
 'C',
 5800,
 20.0,
 20.0]],
 [2593311,
 ['43741-MTL@CAR',
 '2019-10-03',
 2593311,
 'Erik Haula',
 'Carolina Hurricanes',
 'C',
 4600,
 19.5,
 19.5]],
 [5161,
 ['43741-MIN@NSH',
 '2019-10-03',
 5161,
 'Matt Duchene',
 'Nashville Predators',
 'C',
 5600,
 19.5,
 19.5]],
 [3112266,
 ['43742-VGS@SJ',
 '2019-10-04',
 3112266,
 'Tomas Nosek',
 'Vegas Golden Knights',
 'C',
 2600,
 31.3,
 17.9]],
 [2563057,
 ['43742-VGS@SJ',
 '2019-10-04',
 2563057,
 'William Karlsson',
 'Vegas Golden Knights',
 'C',
 5800,
 8.5,
 17.15]],
 [5731,
 ['43741-ARI@ANA',
 '2019-10-03',
 5731,
 'Derek Stepan',
 'Arizona Coyotes',
 'C',
 4800,
 15.8,
 15.8]],
 [4024989,
 ['43741-WPG@NYR',
 '2019-10-03',
 4024989,
 'Brett Howden',
 'New York Rangers',
 'C',
 3100,
 15.6,
 15.6]]]],
 ['W',
 [[3114727,
 ['43740-VAN@EDM',
 '2019-10-02',
 3114727,
 'Leon Draisaitl',
 'Edmonton Oilers',
 'W',
 7000,
 34.0,
 34.0]],
 [2976860,
 ['43740-VAN@EDM',
 '2019-10-02',
 2976860,
 'Tanner Pearson',
 'Vancouver Canucks',
 'W',
 4000,
 29.8,
 29.8]],
 [3899938,
 ['43741-CGY@COL',
 '2019-10-03',
 3899938,
 'Mikko Rantanen',
 'Colorado Avalanche',
 'W',
 6900,
 29.3,
 29.3]],
 [2968772,
 ['43741-MIN@NSH',
 '2019-10-03',
 2968772,
 'Filip Forsberg',
 'Nashville Predators',
 'W',
 6500,
 26.0,
 26.0]],
 [2501107,
 ['43742-TOR@CBJ',
 '2019-10-04',
 2501107,
 'Cam Atkinson',
 'Columbus Blue Jackets',
 'W',
 6700,
 23.8,
 23.8]],
 [2563026,
 ['43742-WPG@NJ',
 '2019-10-04',
 2563026,
 'Blake Coleman',
 'New Jersey Devils',
 'W',
 4000,
 23.0,
 23.0]],
 [3149827,
 ['43741-BUF@PIT',
 '2019-10-03',
 3149827,
 'Conor Sheary',
 'Buffalo Sabres',
 'W',
 4100,
 22.8,
 22.8]],
 [5227,
 ['43741-MTL@CAR',
 '2019-10-03',
 5227,
 'Tomas Tatar',
 'Montreal Canadiens',
 'W',
 4500,
 22.3,
 22.3]],
 [3899937,
 ['43742-TOR@CBJ',
 '2019-10-04',
 3899937,
 'Mitch Marner',
 'Toronto Maple Leafs',
 'W',
 7000,
 33.6,
 21.55]],
 [2590379,
 ['43741-CGY@COL',
 '2019-10-03',
 2590379,
 'Joonas Donskoi',
 'Colorado Avalanche',
 'W',
 3100,
 21.5,
 21.5]],
 [2500990,
 ['43742-VGS@SJ',
 '2019-10-04',
 2500990,
 'Reilly Smith',
 'Vegas Golden Knights',
 'W',
 5600,
 12.8,
 21.4]],
 [3114741,
 ['43742-WPG@NJ',
 '2019-10-04',
 3114741,
 'Nikolaj Ehlers',
 'Winnipeg Jets',
 'W',
 5900,
 8.0,
 20.75]],
 [3961,
 ['43742-WPG@NJ',
 '2019-10-04',
 3961,
 'Blake Wheeler',
 'Winnipeg Jets',
 'W',
 6100,
 6.0,
 20.65]],
 [5556,
 ['43741-MIN@NSH',
 '2019-10-03',
 5556,
 'Austin Watson',
 'Nashville Predators',
 'W',
 2900,
 19.7,
 19.7]],
 [2590389,
 ['43741-FLA@TB',
 '2019-10-03',
 2590389,
 'Ondrej Palat',
 'Tampa Bay Lightning',
 'W',
 3600,
 19.5,
 19.5]],
 [2593315,
 ['43741-MIN@NSH',
 '2019-10-03',
 2593315,
 'Jason Zucker',
 'Minnesota Wild',
 'W',
 4800,
 19.5,
 19.5]],
 [3120307,
 ['43741-MIN@NSH',
 '2019-10-03',
 3120307,
 'Viktor Arvidsson',
 'Nashville Predators',
 'W',
 7400,
 19.5,
 19.5]],
 [5545,
 ['43742-VGS@SJ',
 '2019-10-04',
 5545,
 'Mark Stone',
 'Vegas Golden Knights',
 'W',
 7200,
 19.3,
 17.8]],
 [5428,
 ['43742-WPG@NJ',
 '2019-10-04',
 5428,
 'Taylor Hall',
 'New Jersey Devils',
 'W',
 7500,
 17.5,
 17.5]],
 [2592095,
 ['43741-MTL@CAR',
 '2019-10-03',
 2592095,
 'Teuvo Teravainen',
 'Carolina Hurricanes',
 'W',
 4700,
 17.5,
 17.5]],
 [3114747,
 ['43742-WSH@NYI',
 '2019-10-04',
 3114747,
 'Jakub Vrana',
 'Washington Capitals',
 'W',
 3800,
 14.3,
 16.9]]]],
 ['D',
 [[2562605,
 ['43741-MTL@CAR',